In [1]:
# DS-22 Самощенков Сергей
# Домашнее задание «Продвинутый pandas 2»
# Преподаватель: Константин Башевой

In [28]:
# Задание 1

# Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

import pandas as pd

ratings_data = pd.read_csv('ml-latest-small/ratings.csv')

ratings_data.head(5)

ratings_data = (
    ratings_data[ratings_data['rating'] == 5]
    .groupby('userId').agg(
        time_max = pd.NamedAgg(column = 'timestamp', aggfunc = 'max'),
        time_min = pd.NamedAgg(column = 'timestamp', aggfunc = 'min'),
        rate_count = pd.NamedAgg(column = 'rating', aggfunc = 'count'),
    )
    .sort_values(['userId'], ascending=[True])
    )

# Вариант 1
ratings_data['time_delta_100'] = ratings_data.apply(lambda row: row['time_max'] - row['time_min'] if row['rate_count']>100 else None , axis=1)
print(f"Среднее время жизни пользователей, которые выставили более 100 оценок: {round(ratings_data['time_delta_100'].mean()/3600,1)} час.")

# Вариант 2
ratings_data['time_delta'] = ratings_data.apply(lambda row: row['time_max'] - row['time_min'], axis=1)
print(f"Среднее время жизни пользователей, которые выставили более 100 оценок: {round(ratings_data['time_delta'][ratings_data['rate_count']>100].mean()/3600,1)} час.")




Среднее время жизни пользователей, которые выставили более 100 оценок: 19061.7 час.


In [47]:
# Задание 2

# Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). 
# Необходимо сформировать две таблицы:
# таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
# аналогичную таблицу по типам выручки с указанием адреса клиента
# Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

# К домашнему заданию, задача 2
# Дана статистика услуг перевозок клиентов компании по типам:

# rzd - железнодорожные перевозки
# auto - автомобильные перевозки
# air - воздушные перевозки
# client_base - адреса клиентов

rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)

rzd.head(5)

auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

auto.head(5)

air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

air.head(5)

client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

# Таблица по типам выручки с указанием адреса клиента
client_revenue_with_address = client_base.merge(auto, on='client_id', how='left').merge(air, on='client_id', how='left').merge(rzd, on='client_id', how='left')
# Таблица по типам выручки без указания адреса клиента
client_revenue_without_address = auto.merge(air, on='client_id', how='outer').merge(rzd, on='client_id', how='outer')
# Таблица по типам выручки без указания адреса клиента - вариант 2
client_revenue_without_address_var2 = client_revenue_with_address.loc[:, ['client_id', 'auto_revenue', 'air_revenue', 'rzd_revenue']]
# Таблица по типам выручки без указания адреса клиента - вариант 3
client_revenue_without_address_var3 = pd.DataFrame(client_revenue_with_address, columns=['client_id', 'auto_revenue', 'air_revenue', 'rzd_revenue'])

print('Таблица по типам выручки с указанием адреса клиента', '\n')
print(client_revenue_with_address.head(8))
print('\n', 'Таблица по типам выручки без указания адреса клиента', '\n')
print(client_revenue_without_address.head(8))
print('\n', 'Таблица по типам выручки без указания адреса клиента - вариант 2', '\n')
print(client_revenue_without_address_var2.head(8))
print('\n', 'Таблица по типам выручки без указания адреса клиента - вариант 3', '\n')
print(client_revenue_without_address_var3.head(8))



Таблица по типам выручки с указанием адреса клиента 

   client_id          address  auto_revenue  air_revenue  rzd_revenue
0        111  Комсомольская 4           NaN          NaN       1093.0
1        112   Энтузиастов 8а           NaN          NaN       2810.0
2        113  Левобережная 1а       57483.0          NaN      10283.0
3        114          Мира 14          83.0          NaN       5774.0
4        115        ЗЖБИиДК 1         912.0         81.0        981.0
5        116    Строителей 18        4834.0          4.0          NaN
6        117  Панфиловская 33          98.0         13.0          NaN
7        118     Мастеркова 4           NaN        173.0          NaN

 Таблица по типам выручки без указания адреса клиента 

   client_id  auto_revenue  air_revenue  rzd_revenue
0        113       57483.0          NaN      10283.0
1        114          83.0          NaN       5774.0
2        115         912.0         81.0        981.0
3        116        4834.0          4.0        

In [ ]:
# Задание 3

# В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. 

# Как бы вы добавили эти сведения в таблицу визитов и покупок? 

1. для каждого визита нужно взять координату (x1; y1) за ближайшее время (t1) до времени визита (t) и координату (x2; y2) за ближайшее время после визита (t2)
2. записать в таблицу визитов координату по формуле (x1 + (x2-x1)*(t-t1)/(t2-t1); y1 + (y2-y1)*(t-t1)/(t2-t1))

# Для составления ответа можно использовать вопросы:
# У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
# Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
точность расчетной координаты уменьшится 
# Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.
маршрут клиента, адреса: индекс, город, улица, дом
